<a href="https://colab.research.google.com/github/sgr1118/GPT-/blob/main/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT(Generative Pre-Training Transformer)

GPT는 트랜스포머의 decoder 구조만을 이용하여 만든 네트워크입니다. 트랜스포머의 decoder를 아주 깊고 깊게 쌓아 많은 데이터를 학습 시켜 성능을 높힌 네크워크다.

1. GPT의 구조

Decoder만을 이용했다는 것을 이해하기위해 트랜스포머의 Decoder를 먼저 봐야할 필요가 있다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/10_egROfLK.png)
<center>트랜스포머 Deocder</center>

- Decoder는 masked Multi-Head Attention, Multi-Head Attention, Feed Forward Neural Network로 이루어져 있다. 바로 이 구조를 차용하여 Decoder block를 많이 쌓아 올리면 GPT가 된다.

GPT를 두 부분으로 나누어서 좀 더 자세히 보면 아래와 같습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/11_HNGuRhf.png)
<center>GPT의 모델 구조</center>

Transfomer Decoder Block : Pretraining LM (Unspervised Learning)

- 첫 번째로 볼 부분은 빨간색 박스 안이다. pretrain 하는 부분으로 unsupervised learning(비지도 학습)을 하게 됩니다. 위에 트랜스포머의 구조 그대로다

1. Embedding
- GPT는 텍스트의 Embedding으로 BPE(Byte-pair Encoding)를 사용하고 있다.
- BPE는 모든 단어를 문자(바이트)들의 집합으로 취급하여 자주 등장하는 문자 쌍을 합치는 subword tokeniztion이었다. 처음보는 단어일지라도 문자(알파벳)들의 조합으로 나타내어 OOV 문제를 해결할 수 있다는 장점이 있다. 기존 트랜스포머와 마찬가지로 position encoding(포지션 인코딩)도 함께 사용한다.

2. Masked Multi-Head Attention
- Masked Multi-Head Attention은 모든 것을 병렬적으로 처리하는 트랜스포머에 자기회귀적인(Autoregressive)인 특성을 부여하기 위해 만든 장치이다. 여기서 자기회귀적이라 함은 훈련 단계에서 디코더에게 정답 문장을 매 스텝 단위로 단어 하나씩 알려주고 그 다음 단어를 예측(Next Token Prediction)하게 하는 형태로 학습되는 형태하는 뜻이다. 이는 마치 sequence-to-sequence 모델에서 디코더가 번역 문장을 생성할 때 time-step을 하나하나 거치듯이 만들어주는 것입니다. 순차처리 방식의 RNN과 달리 정답 문장의 모든 단어를 한꺼번에 입력받는 트랜스포머의 deocder는 학습할 때 현재 자기보다 미래에 생성될 토큰을 보지 못하도록 masking이 필요하게 됩니다.

- 이 구조는 우리가 학습했던 언어 모델과 같은 구조입니다. 다시 한번 되짚어 보자면, 언어 모델은 비지도 학습을 통해 문장의 자연스러운 순서를 학습하게 됩니다. 그래서 GPT는 문자 생성에 매우 특화되어 있다. GPT를 연구한 Open Ai도 너무나 자연스러운 문장을 만들어내서 악용이 우려하고 있다.

- Text Prediction & Text classification: finetuning downstream task (Supervised Learning)

- pretrain이 끝나게 되면 GPT는 downstream task에 맞게 finetuning을 하게 됩니다. 바로 파란색 박스 부분에 해당되는 부분이다. 여기서 우리는 기존에 봤던 모델들과 조금 다른 점을 발견할 수 있다. 바로 두개의 Objective가 존재한다는 것이다.

- 말 그대로 모델이 두 가지의 문제를 동시에 푸는 겁니다. text prediction과 text classification이 각각 다른 모델들을 이용하여 output을 만들어내는 것이 아니라 한 모델에서 동시에 output을 내는 겁니다.

- 논문의 저자들은 이렇게 실제로 하는 문제인 주요 task와 동시에 보조적으로 또 다른 문제를 풀 때 (Auxiliary objective)주요 task에 대한 정확도가 더 올라갔음을 확인했다.

- 생각해보면 LM또한, auxiliary로 얻어진 결과라고 생각할 수 있습니다. 시퀀스의 다음 나올 단어들을 학습하다 보니 전체적인 언어의 구조를 알게 된 것이다.

- GPT의 전체 코드는 약간 복잡해 보일 수 있지만, TFGPT2MainLayer라는 전체 모델 클래스 안에서 TFBlock 레이어 클래스를 반복해서 사용하고 있는 부분을 잘 봐야한다. TFBlock 클래스 안에서 TFAtttion, TFMLP 레이어가 사용되는 구조가 위에서 소개한 GPT의 모델 구조 그림에 표현되어 있다.

Input Transformation

- 모델이 한 개인데 어떻게 classification, entailment 등등 다양한 문제를 풀 수 있을까요?

![](https://d3s0tskafalll9.cloudfront.net/media/images/12_m2lKyFg.max-800x600.png)
<center>GPT task</center>

- 바로 input을 변형시켜서 입니다.

- GPT는 언어 모델이기 때문에 pretrain시에는 문장(단어의 시퀀스들)을 그대로 input을 줍니다. finetuning시에도 똑같이 단어의 시퀀스들을 주면 된다. 이때, input을 아주 조금만 변형시켜 주면 됩니다.

- 예를 들어 classification task를 풀기 위해 finetuning을 하게 된다면 start, input text, extract, class 이렇게 구성된 데이터셋을 학습시키면 된다. GPT는 이 데이터셋에 맞추어 weight들을 조정하게 될 것이다.

- finetuning이 끝나고 테스트 시에는  start, input text, extract을 input으로 주면 해당 시퀀스에 뒤이어 나올 토큰 즉 class를 생성하게 되는 것이다.

# Pytorch를 기반으로한 GPT Model 구현

In [ ]:
import torch
import torch.nn as nn

### 1. Config
- Transformer와 파라미터를 동일하게 설정
- GPT는 Decoder만 사용하므로 Encoder 부분은 제거한다.
- [config.json](https://github.com/paul-hyun/transformer-evolution/blob/master/gpt/config.json)

In [ ]:
config = Config({
    "n_dec_vocab": len(vocab),
    "n_dec_seq": 256,
    "n_layer": 6,
    "d_hidn": 256,
    "i_pad": 0,
    "d_ff": 1024,
    "n_head": 4,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12
})
print(config)

NameError: ignored

### 2. Decoder

![](https://paul-hyun.github.io/assets/2019-12-30/decoder.png)
<center>Decoder</center>

- GPT는 표준 Transformer의 Encoder는 사용하지 않고 Decoder만 사용하므로 Decoder에서 Encoder의 출력과 Attention을 하는 부분인 Encoder-Decoder Multi-Head Attention 부분을 제거해야 한다. 나머지 부분은 Transformer와 동일하다.


### Decoder Layer
- 표준 Transformer DecoderLayer에서 Encoder-Decoder Multi-Head Attention을 제거한 코드 입니다.

In [ ]:
""" decoder layer """
class Decoderlayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.self_attn = MultiheadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps = self.config.layer_norm_epsilon)
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm3 = nn.LayerNorm(self.config.d_hidn, eps = self.config.layer_norm_epsilon)

    def forward(self, dec_inputs, self_attn_mask):
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq)
        self_att_outputs, self_attn_prob = self.self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask)
        self_att_outputs = self.layer_norm1(dec_inputs + self_att_outputs)
        # (bs, n_dec_seq, d_hidn)
        ffn_outputs = self.pos_ffn(self_att_outputs)
        ffn_outputs = self.layer_norm3(self-att_outputs + ffn_outputs)
        # (bs, n_dec_seq, d_hidn), (bs, n_head, n_dec_seq, n_dec_seq), (bs, n_head, n_dec_seq, n_enc_seq)
        return ffn_outputs, self_attn_prob    

### Decoder
- 표준 Transformer Decoder에서 Encoder출력을 DecoderLayer에 입력하는 부분을 제거한 코드 입니다.

In [ ]:
""" decoder """
class Decoder(nn.Module):
    def __init__(self, config):
        super().__init()
        self.config = config

        self.dec_emb = nn.Embedding(self.config.n_dec_vocab, self.config.d_hidn)
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config.n_dec_seq + 1, self.config.d_hidn))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)

        self.layers = nn.ModuleList([DecoderLayer(self.config) for _ in range(self.config,n_layer)])
    
    def forward(self, dec_inputs):
        positions = torch.arange(dec_inputs.size(1), device = dec_inputs.device, dtype = dec_inputs.dtype).expand(dec_inputs.size(0), dec_inputs.size(1)).contiguous() + 1
        pos_mask = dec_inputs.eq(self.config.i_pad)
        positions.masked_fill_(pos_mask, 0)

        # (bs, n_dec_seq, d_hidn)
        dec_outputs = self.dec_emb(dec_imputs) + self.pos_emb(positions)

        # (bs, n_dec_seq, n_dec_seq)
        dec_attn_pad_mask = get_attn_apd_mask(dec_inputs, dec_inputs, self.config.i_pad)
        # (bs, n_dec_seq, n_dec_seq)
        dec_attn_decoder_mask = get_attn_decoder_mask(dec_inputs)
        # (bs, n_dec_seq, n_dec_seq)
        dec_self_attn_mask = torch.gt((dec_attn_pad_mask + dec_attn_decoder_mask), 0)

        self_attn_probs = []
        for layer in self.layers:
            # (bs, n_dec_seq, d_hidn), (bs, n_dec_seq, n_dec_seq)
            dec_inputs, self_attn_prob = layer(dec_outputs, dec_self_attn_mask)
            self_attn_probs.append(self_attn_prob)
        # (bs, n_dec_seq, d_hidn), [(bs, n_dec_seq, n_dec_seq)]
        return dec_outputs, self_attn_probs

### 3. GPT

GPT는 단순히 Transformer Decoder를 실행합니다.
Pretrain된 모델을 저장하기위한 save, 저장된 모델을 읽기위한 load 함수가 추가로 정의 되었습니다

In [ ]:
"""GPT"""

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.decoder = Decoder(self.config)

    def forward(self, dec_inputs):
        # (bs, n_seq, d_hidn), [(bs, n_head, n_dec_seq, n_dec_seq)]
        dec_outputs, dec_self_attn_probs = self.decoder(dec_inputs)
        # (bs, n_dec_seq, n_dec_vocab), [(bs, n_head, n_dec_seq, n_dec_seq)]
        return dec_outputs, dec_self_attn_probs

    def save(self, epoch, loss, path):
        torch.save({
            "epoch" : epoch,
            "loss" : loss,
            "state_dict" : self.state_dict()
        }, path)

    def load(self, path):
        save = torch.load(path)
        self.load_state_dict(save["state_dict"])
        return save["epoch"], save["loss"]

### 4. Pretrain Model

![](https://paul-hyun.github.io/assets/2019-12-30/pretrain.png)
<center></center>

GPT를 Pretrain 하기위한 클래스 입니다.
GPTPretrain 클래스의 목적은 입력 단어에 대한 다음 단어를 예측 하는 겁니다.

1. GPT의 결과를 입력으로 단어를 예측하기 위한 projection_lm를 선업합니다. (줄 : 10)

2. projection_lm은 Decoder의 Embedding과 weight를 share합니다. (줄 : 11)

3. GPT 실행 결과를 입력으로 projection_lm을 실행해서 단어를 예측하도록 합니다. (줄 : 17)

4. 입력에 대한 다음 단어를 예측하는 것이므로 결과의 마지막을 제외한 나머지를 리턴합니다. (줄 : 19)


In [ ]:
""" GPT pretrain """

class GPTPretrain(nn.Module):
    def __init__(self, config):
        super().__init()
        self.config = config

        self.gpt = GPT(self.conig)
        # lm
        self.projection_lm = nn.Linear(self.config.d_hidn, self.config.n_dec_vocab, bias=False)
        self.projection_lm.weight = self.gpt.decoder.dec_emb.weight

    def forward(self, dec_inputs):
        # (bs, n_dec_seq, d_hidn), [(bs, n_head, n_dec_seq, n_dec_seq)]
        dec_outputs, dec_self_attn_probs = self.gpt(dec_inputs)
        # (bs, n_dec_seq, n_dec_vocab)
        logits_lm = self.projection_lm(dec_outputs)
        # (bs, n_dec_seq - 1, n_dec_vocab), (bs, n_output), [(bs, n_head, n_dec_seq, n_dec_seq)]
        return logits_lm[:, :-1, :].contiguous(), dec_self_attn_probs

### 5. Pretrain Data 생성

단락별 pretrain 데이터 생성 함수
단락을 여러 개의 Pretrain 데이터로 만드는 함수 입니다.

1. 입력의 시작은 [BOS] 끝은 [EOS]입니다. tgt_seq는 n_seq에서 2개를 뺀 값입니다. (줄 : 5, 6)

2. 단락을 줄 단위로 for loop를 돌며 아래내용 (줄 : 4 ~ 7)을 실행합니다. (줄 : 11)

3. current_chunk에 line을 추가, current_length에 라인의 tokens 수를 더합니다. (줄 : 12, 13)

4. 마지막 줄 이거나 current_length가 tgt_seq를 넘을 경우 학습데이터를 만듭니다. (줄 : 14)

5. current_chunk의 값을 tokens로 만들고 tgt_seq를 초과하는 부분은 제거합니다. (줄 : 17, 18)

6. [BOS] + tokens + [EOS] 형태로 데이터를 생성합니다. (줄 : 21)

In [ ]:
""" doc별 pretrain 데이터 생성 """

def create_pretrain_instances(doc, n_seq):
    # for [BOS], [EOS]
    max_seq = n_seq - 2
    tgt_seq = max_seq

    instances = []
    current_chunk = []
    current_lenght = []
    for i in range(len(doc)):
        current_chunk.append(doc[i]) # line
        current_lenght += len(doc[i])
        if i == len(doc) - 1 or current_lenght >= tgt_seq:
            if 0 < len(current_chunk):
                tokens = []
                for chunk in current_chunk: tokens.extend(chunk)
                tokens = tokens[:tgt_seq]
                if 1 < len(tokens):
                    instance = {
                        "tokens" : ["[BOS]"] + tokens + ["[EOS]"],
                    }
                    instances.append(instance)
            current_chunk = []
            current_lenght = 0 
    return instances

pretrain 데이터 생성 함수
말뭉치를 읽어 Pretrain 데이터를 만드는 함수 입니다.

In [ ]:
""" pretrain 데이터 생성 """

def make_pretrain_data(vocab, in_file, out_file, n_seq):
    line_cnt = 0
    with open(in_file, "r") as in_f:
        for line in in_f:
            line_cnt += 1

    docs = []
    with open (in_file, "r") as f:
        doc = []
        with tqdm(total = line_cnt, desc = f"Lodaing") as pbar:
            for i, line in enumerate(f):
                line = line.strip()
                if line == "":
                    if 0 < len(doc):
                        docs.append(doc)
                        doc = []
                else:
                    pieces = vocab.encode_as_pieces(line)
                    if 0 < len(pieces):
                        doc.append(pieces)
                    